In [1]:
import pandas as pd
import numpy as np

def monthly_rebalancing(daily_returns, weights, initial_dollar):
    
    """
    Calculate the returns for a portfolio rebalanced monthly.
    
    Parameters:
    daily_returns (DataFrame): A pandas dataframe with the daily returns of the securities in the portfolio.
    weights (dict): A dictionary with the weights of each security in the portfolio.
    initial_dollar (float): The initial capital for the portfolio. Default is 100.
    
    Returns:
    DataFrame: A pandas dataframe with the cumulative returns for the rebalanced portfolio.
    """
    
    # Group the dataframe by month using the .groupby() method
    grouped = daily_returns.groupby(pd.Grouper(freq='M'))

    # Split the dataframe into monthly dataframes
    monthly_dfs = {name: group for name, group in grouped}
    
    monthly_index_dfs = []
    #Loop through list of monthly dataframes to rebalance the portfolio
    for i in list(monthly_dfs.keys()):
        monthly_index = monthly_dfs[i].copy()
        #Set first month values based on initial capital and weights
        if i == list(monthly_dfs.keys())[0]:
            monthly_value = weights.copy()
            for key in monthly_value:
                monthly_value[key] = monthly_value[key] * initial_dollar
        #Set the values of first day of each month to last month's total value
        else:
            monthly_value = weights.copy()
            for key in monthly_value:
                monthly_value[key] = monthly_value[key] * monthly_total_value

        #Set first value of each month according to rebalancing to calculate return series
        monthly_index.iloc[0] = monthly_value
        monthly_index.iloc[1:] = monthly_index.iloc[1:] + 1
        monthly_index = monthly_index.cumprod()
        monthly_index_dfs.append(monthly_index)
        
        #Get total value at end of month for next month rebalancing
        monthly_total_value = monthly_index.iloc[-1].sum()
        
    rebal_port_index = pd.concat(monthly_index_dfs)
        
    return rebal_port_index

def backtest_returns(prices, weights, initial_dollar=100):
    
    """
    Backtest the returns for a portfolio based on set weights rebalanced at a certain frequency.
    For now its fixed at monthly.
    
    Parameters:
    prices (DataFrame): A pandas dataframe with the daily prices of the securities in the portfolio.
    weights (dict): A dictionary with the weights of each security in the portfolio.
    initial_dollar (float): The initial capital for the portfolio. Default is 100.
    
    Returns:
    DataFrame: A pandas dataframe with the cumulative returns for the rebalanced portfolio.
    """
    
    # Check if the weights sum to 1
    weight_sum = sum(weights.values())
    if not np.isclose(weight_sum, 1.0, rtol=1e-3):
        raise ValueError("The sum of weights must be equal to 1.")
    
    # Check if all tickers in the weights dictionary are also in the prices dataframe
    if not set(weights.keys()).issubset(set(prices.columns)):
        raise ValueError("Not all tickers in the weights dictionary are also in the prices dataframe.")
    
    # Get the daily percentage changes for the price data
    daily_returns = prices.copy()
    daily_returns = daily_returns.pct_change()
    
    rebalanced_index = monthly_rebalancing(daily_returns, weights, initial_dollar)
    
    rebalanced_index['Total'] = rebalanced_index.sum(axis=1)
    
    return rebalanced_index

The given code is a python script for backtesting a portfolio's returns. The script consists of two main functions, `monthly_rebalancing` and `backtest_returns`, and a piece of code at the bottom which reads in a `.csv` file of daily prices and sets the weights for the portfolio. 

## Function 1: `monthly_rebalancing`

The `monthly_rebalancing` function takes in three parameters:
- `daily_returns`: a pandas dataframe with the daily returns of the securities in the portfolio.
- `weights`: a dictionary with the weights of each security in the portfolio.
- `initial_dollar`: the initial capital for the portfolio. The default value is 100.

The function returns a pandas dataframe with the cumulative returns for the rebalanced portfolio.

The code works as follows:
1. The daily returns dataframe is grouped by month using the `.groupby()` method.
2. The grouped dataframe is split into separate dataframes, each corresponding to a single month.
3. The script loops through the list of monthly dataframes and performs the following steps for each month:
   - Initialize the values of each security in the portfolio according to the given weights and the initial capital.
   - For first day in the month, the values of each security in the portfolio are rebalanced.
       - If its the first day of the first month, it takes weights multiplied by initial capital.
       - If its subsequent months it takes the total portfolio value at end of last month multiplied by weights
   - The cumulative return for each security is calculated by taking the cumulative product of the daily returns.
   - The total value of the portfolio is calculated as the sum of the values of each security.
4. The cumulative returns for each month are concatenated into a single dataframe, which is returned as the final result.

## Function 2: `backtest_returns`

The `backtest_returns` function takes in three parameters:
- `prices`: a pandas dataframe with the daily prices of the securities in the portfolio.
- `weights`: a dictionary with the weights of each security in the portfolio.
- `initial_dollar`: the initial capital for the portfolio. The default value is 100.

The function returns a pandas dataframe with the cumulative returns for the rebalanced portfolio.

The code works as follows:
1. The script checks if the sum of the weights in the `weights` dictionary is close to 1.0. If the sum is not close to 1.0, a `ValueError` is raised.
2. The script checks if all the tickers in the `weights` dictionary are also present in the `prices` dataframe. If not, a `ValueError` is raised.
3. The daily returns are calculated as the daily percentage changes in the prices of each security.
4. The `monthly_rebalancing` function is called with the daily returns, the `weights`, and the `initial_dollar` as parameters. The result is stored in a variable called `rebalanced_index`.
5. The total returns for the portfolio are calculated as the sum of the returns of each security in the portfolio.
6. The `rebalanced_index` dataframe is returned as the final result.

## Code at the bottom

At the bottom of the script, a `.csv` file called `daily_prices.csv` is read in and stored in a variable called `prices`. The weights for the portfolio are set as a dictionary, with `'SPX Index'` having a weight of 0.5 and `'ND


In [2]:
prices = pd.read_csv("daily_prices.csv", index_col=0, parse_dates=True)
weights = {'SPX Index': 0.5, 'NDX Index': 0.5}
returns = backtest_returns(prices, weights)

In [3]:
prices

,SPX Index,NDX Index
Dates,,
2022-11-01,3856.10,11288.95
2022-11-02,3759.69,10906.34
2022-11-03,3719.89,10690.60
2022-11-04,3770.55,10857.03
2022-11-07,3806.80,10977.00
...,...,...
2023-02-07,4164.00,12728.27
2023-02-08,4117.86,12495.38
2023-02-09,4081.50,12381.17


In [4]:
returns

,SPX Index,NDX Index,Total
Dates,,,
2022-11-01,50.000000,50.000000,100.000000
2022-11-02,48.749903,48.305378,97.055281
2022-11-03,48.233837,47.349842,95.583679
2022-11-04,48.890719,48.086979,96.977697
2022-11-07,49.360753,48.618339,97.979092
...,...,...,...
2023-02-07,53.824761,54.818522,108.643283
2023-02-08,53.228346,53.815504,107.043849
2023-02-09,52.758348,53.323620,106.081969
